In [1]:
import os
import re
import nltk
from pdfminer.high_level import extract_pages
from pdfminer.high_level import LTPage
from nltk.corpus import wordnet as wn
from pdfminer.layout import LTTextContainer, LTTextLineVertical, LTTextBoxHorizontal, LTTextBox, Rect
import wordninja
from pdf2image import convert_from_path
import numpy as np
import cv2
import pandas as pd
import pickle

In [2]:
def identify_footnote_line(article_path: str, page: LTPage, page_number: int, DPI: int = 100) -> list[dict[str, float]]:
    page_w, page_h = page.bbox[2], page.bbox[3]
    raw_image = convert_from_path(article_path, dpi=DPI, first_page=page_number+1, last_page=page_number+1)[0]
    img = np.array(raw_image)
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_height, img_width = gray.shape
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
    morph = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    edges = cv2.Canny(morph, 50, 200, apertureSize=3)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=20)
    if lines is None:
        return None

    pdfminer_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Escalar a coordenadas PDFMiner
        y1_pdf = (y1 / img_height) * page_h
        y2_pdf = (y2 / img_height) * page_h

        pdfminer_lines.append({
            "x1": x1,
            "y1": page_h - y1_pdf,
            "x2": x2,
            "y2": page_h - y2_pdf
        })

    threshold_y = page_h * 0.35
    filtered_lines = [line for line in pdfminer_lines if line["y1"] <= threshold_y]
    filtered_lines.sort(key=lambda line: line["y1"], reverse=True)
    return filtered_lines[0] if filtered_lines else None

In [3]:
def extract_page(pdf_path, page):
    pages = extract_pages(pdf_path)
    for i, page_layout in enumerate(pages):
        if i != page - 1: continue
        return page_layout
    return None

In [4]:
# regex stop conditions
references_re = r"(?i)^\s*(referencias|bibliografía citada|bibliografia citada|bibliografía|bibliografia|citas|fuentes|referencias bibliograficas|referencias bibliográficas)\s*$"

# regex delete
version_logs = r"(?i)(recibido|aceptado|publicado|(segunda|tercera|cuarta|quinta|sexta) versión)[,:]?\s*(el\s*)?(\d{1,2}(\s*de)?\s*[a-záéíóú]+(\s*de\s*\d{4})?|\d{1,2}/\d{1,2}/\d{4})(\s*\.\s*)?"

footnote_references = r"([.,;:!?])([0-9]+)"
duplicated_spaces = r"\s+"

# recognizing regex
abstract = r"Palabras clave|Keywords|Resumen|Abstract|Key words|Palabrasclave|Keyword|Palabra clave|Palabras claves|Keywords|Sumario|Síntesis|Sintesis|Sinopsis"

# parentesis y llaves
parentheses_pattern = r'\(' + \
                      r'(?:' + \
                      r'(?:[^;)]+)' + \
                      r'(?:; [^;)]+)*' + \
                      r')\)'
brackets_pattern = r'\[' + \
                      r'(?:' + \
                      r'(?:[^;\]]+)' + \
                      r'(?:; [^;\]]+)*' + \
                      r')\]'

In [5]:
class ArticleExtractedData:
    def __init__(self, pdf_path: str):
        self.pdf_path: str = pdf_path
        self.headers: set[str] = set()
        self.footers: set[str] = set()
        self.footnotes_lines: list[dict[str, float]] = []

def extract_data_article(pdf_path) -> ArticleExtractedData:
    data = ArticleExtractedData(pdf_path)
    headers_counter: map[str, int] = {}
    footers_counter: map[str, int] = {}

    for i, page in enumerate(extract_pages(pdf_path)):
        height = page.bbox[3]
        cutoff = height * 0.35

        footnote_line = identify_footnote_line(pdf_path, page, i)
        data.footnotes_lines.append(footnote_line)

        for element in page:
            if not isinstance(element, (LTTextBox, LTTextBox, LTTextContainer)):
                continue
            txt = element.get_text().strip()
            if element.bbox[3] <= cutoff:
                footers_counter[txt] = footers_counter.get(txt, 0) + 1
            if element.bbox[1] >= height - cutoff:
                headers_counter[txt] = headers_counter.get(txt, 0) + 1


    data.headers = {k for k, v in headers_counter.items() if v > 1 and len(k) > 0}
    data.footers = {k for k, v in footers_counter.items() if v > 1 and len(k) > 0}
    return data

In [6]:
def check_stop_conditions(text) -> bool:
	return any([
		re.search(references_re, text, flags=re.MULTILINE),
	])

def clean_unwanted_patterns(text) -> str:
	# Eliminar patrones como "Recibido el 12 de marzo de 2021."
    text = re.sub(version_logs, "", text).strip()

    # ()
    text = re.sub(parentheses_pattern, "", text).strip()
    
    # []
    text = re.sub(brackets_pattern, "", text).strip()
    
    # .1 ?2 !4 :5 ;6
    text = re.sub(footnote_references, r"\1", text).strip()

    return text

def clean_page(text) -> str:
    text = text.replace("-\n", "")
    text = text.replace("\n", " ")
    text = text.replace("«", "").replace("»", "").replace("“", "").replace("”", "").replace("‘", "").replace("’", "")
    text = text.replace('"', "").replace("'", "")
    # en ., data ?
    text = re.sub(r'\s+([.,;:!?])', r'\1', text).strip()
    text = re.sub(duplicated_spaces, " ", text).strip()
    return text.lower()

# (str, bool) -> (text, continue?)
def extract_page_text(page: LTPage, page_number: int, data: ArticleExtractedData) -> tuple[str, bool]:
    text = ""
    status = True

    for element in page:
        if not isinstance(element, (LTTextBoxHorizontal, LTTextBox, LTTextContainer)):
            continue

        _, __, ___, y2 = element.bbox
        footnote_line = data.footnotes_lines[page_number]
        if y2 <= (footnote_line["y1"] if footnote_line is not None else -1):
            break

        txt = element.get_text().strip()
        if txt.isdigit() or txt in data.headers or txt in data.footers:
            continue

        if check_stop_conditions(txt):
            status = False
            break

        text += txt + "\n"

    text = clean_unwanted_patterns(text)
    text = clean_page(text)
    return text, status

# Preprocesado

In [7]:
with open("initial_texts-all-articles.pkl", "rb") as f:
    initial_texts: dict[str, str] = pickle.load(f)
len(initial_texts)

1912

In [8]:
def get_files() -> list[str]:
    PACK_DIR = "pack"
    os.makedirs(PACK_DIR, exist_ok=True)
    dirs = os.listdir(PACK_DIR)
    files = []
    for dr in dirs:
        files.extend(
            [
                os.path.join(PACK_DIR, dr, file)
                for file in os.listdir(os.path.join(PACK_DIR, dr))
            ]
        )
    return files

In [9]:
# articles = [os.path.join("data", art) for art in os.listdir("data")]
articles = get_files()
len(articles)

1912

In [19]:
def delete_before_substr(text, substring):
    split = text.split(substring, 1)
    if len(split) == 1:
        return None
    
    return (substring + split[1]).strip()

def extract_pdf_text(pdf_path: str, initial_text: str) -> str:
    text = ""
    data = extract_data_article(pdf_path)
    for i, page in enumerate(extract_pages(pdf_path)):
        page_text, can_continue = extract_page_text(page, i, data)
        text = f"{text} {page_text}"
        if not can_continue:
            break

    if initial_text is None:
        return text.strip()
    
    initial_substr = clean_page(clean_unwanted_patterns(initial_text.strip()))
    new_text = delete_before_substr(text, initial_substr)
    if new_text is None:
        print(f"{'=' * 20}\nINITIAL TEXT NOT FOUND: {pdf_path}\n{'=' * 20}")
        return None    
    return new_text.strip()

In [20]:
import multiprocessing as mp
from multiprocessing.pool import ThreadPool

def process_article(article, ignore_initial_text=False):
    return article, extract_pdf_text(article, initial_texts[article] if not ignore_initial_text else None)

In [11]:
# articles_text = {}
# for article in articles:
#     articles_text[article] = extract_pdf_text(article)

def process_articles_parallel(articles):
    with ThreadPool(mp.cpu_count() * 8) as pool:
        results = pool.map(process_article, articles)
    return dict(results)

articles_text = process_articles_parallel(articles)

The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Hacia-una-producción-enriquecida-Trabajo-en-grupo-y-recualificación-en-empresas-de-ingeniería-mecáni.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


INITIAL TEXT NOT FOUND: pack\AIBR\2007-Pidiendo-la-defensa-de-un-título-de-doctor-16-años-después.pdf


The PDF <_io.BufferedReader name='pack\\AIBR\\2006-El-trabajo-de-las-mujeres-en-la-industria-maquiladora-de-México-Balance-de-cuatro-décadas-de-estudio.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-caso-de-Bancolombia.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Profesionales-flexibles-Cocineros-enfermeras-y-directivos-en-el-siglo-XXI.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


INITIAL TEXT NOT FOUND: pack\Anuario-de-Estudios-Americanos\2008-La-perspectiva-católica-sobre-la-salud-y-la-práctica-médica-en-la-Argentina-de-los-años-treinta-La-v.pdf
INITIAL TEXT NOT FOUND: pack\Anuario-de-Estudios-Americanos\2007-Centralización-o-descentralización-gestión-pública-o-privada-de-un-bien-escaso-historiografía-sobre-.pdf
INITIAL TEXT NOT FOUND: pack\Anuario-de-Estudios-Americanos\2006-Religiosidad-e-identidad-en-San-Francisco-de-Campeche-Siglos-XVI-y-XVII.pdf
INITIAL TEXT NOT FOUND: pack\Anuario-de-Estudios-Americanos\2003-Políticas-de-la-justicia-criminal-interétnica-en-Córdoba-del-Tucumán-siglos-XVI-y-XVII.pdf
INITIAL TEXT NOT FOUND: pack\Anuario-de-Estudios-Americanos\2006-Economía-rentas-y-situados-en-Cartagena-de-Indias-17611800.pdf
INITIAL TEXT NOT FOUND: pack\AIBR\2018-Humanidad-territorializada-Madres-dueños-y-personas-que-cuidan.pdf
INITIAL TEXT NOT FOUND: pack\Anuario-de-Estudios-Americanos\2007-Las-rutas-del-libro-atlántico-libros-enviados-en-el-navío-de-Hon

The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-caso-de-Bancolombia.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Consecuencias-personales-en-la-ruptura-de-la-vida-laboral-El-caso-de-Telefónica.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Hacia-una-producción-enriquecida-Trabajo-en-grupo-y-recualificación-en-empresas-de-ingeniería-mecáni.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this 

...

INITIAL TEXT NOT FOUND: pack\Revista-de-Indias\2010-Indígenas-andinos-en-Chile-Colonial-Inmigración-inserción-espacial-integración-económica-y-movilidad.pdf


The PDF <_io.BufferedReader name='pack\\Revista-de-Indias\\2006-Mario-de-Andrade-y-laMissão-de-Pesquisas-Folclóricas1938-una-etnografía-que-no-fue.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\Revista-de-Indias\\2006-El-rol-de-las-alianzas-entre-misioneros-e-indígenas-en-la-conquista-de-Apolobamba-siglos-XVIXVII.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\Revista-de-Indias\\2006-Como-güelfos-y-gibelinos-los-colegios-de-San-Bernardo-y-San-Antonio-Abad-en-el-Cuzco-durante-el-sigl.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable 

INITIAL TEXT NOT FOUND: pack\Revista-de-Indias\2008-Debates-en-torno-al-liberalismo-representación-e-instituciones-en-el-Congreso-Constituyente-mexicano.pdf
INITIAL TEXT NOT FOUND: pack\Revista-de-Indias\2011-Un-coloso-sobre-la-arena-definiendo-el-camino-hacia-la-plantación-esclavista-en-Cuba-17921825.pdf
INITIAL TEXT NOT FOUND: pack\Revista-de-Indias\2010-Matrimonio-transgresión-y-conflicto-en-la-región-de-Concepción-Chile-en-el-siglo-XIX.pdf


The PDF <_io.BufferedReader name='pack\\Revista-de-Indias\\2006-Estados-clases-y-Real-Hacienda-en-los-inicios-de-la-conquista-del-Perú.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\Revista-de-Indias\\2006-Historia-Memoria-la-construcción-de-las-tradiciones-dinásticas-andinas.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


INITIAL TEXT NOT FOUND: pack\Revista-Española-de-Antropología-Americana\2009-Mito-conversiones-y-poder-como-carisma-cristiano-entre-los-wichí.pdf
INITIAL TEXT NOT FOUND: pack\Revista-Española-de-Antropología-Americana\2003-José-Alcina-Franch-y-los-códices-mesoamericanos.pdf
INITIAL TEXT NOT FOUND: pack\Revista-Española-de-Antropología-Americana\2009-Prácticas-mortuorias-entre-las-poblaciones-Aguada-del-valle-de-Ambato-Catamarca-Argentina.pdf
INITIAL TEXT NOT FOUND: pack\Revista-Española-de-Antropología-Americana\2009-Más-rasgos-religiosos-de-los-mayas-y-los-egipcios.pdf
INITIAL TEXT NOT FOUND: pack\Revista-de-Indias\2012-Museología-subalterna-sobre-las-ruinas-de-Moctezuma-II.pdf
INITIAL TEXT NOT FOUND: pack\Revista-de-Indias\2010-Elites-locales-y-economía-política-en-la-Mesoamérica-posclásica-el-caso-de-Molango-Señorío-de-Metzti.pdf
INITIAL TEXT NOT FOUND: pack\Revista-de-Indias\2015-De-arrochelados-a-vecinos-reformismo-borbónico-e-integración-política-en-las-gobernaciones-de-Santa-.pd

In [129]:
def extract_file_paths(text):
    pattern = r'INITIAL TEXT NOT FOUND: (.+?)\n'
    matches = re.findall(pattern, text)
    decoded_matches = [match.encode('latin1').decode('utf-8', 'ignore').strip() for match in matches]
    return decoded_matches

with open("initial-text-not-found.log", "r") as f:
    logs = f.read()
not_healthy_files = extract_file_paths(logs)
not_healthy_files

['pack\\AIBR\\2007-Pidiendo-la-defensa-de-un-título-de-doctor-16-años-después.pdf',
 'pack\\Anuario-de-Estudios-Americanos\\2008-La-perspectiva-católica-sobre-la-salud-y-la-práctica-médica-en-la-Argentina-de-los-años-treinta-La-v.pdf',
 'pack\\Anuario-de-Estudios-Americanos\\2007-Centralización-o-descentralización-gestión-pública-o-privada-de-un-bien-escaso-historiografía-sobre-.pdf',
 'pack\\Anuario-de-Estudios-Americanos\\2006-Religiosidad-e-identidad-en-San-Francisco-de-Campeche-Siglos-XVI-y-XVII.pdf',
 'pack\\Anuario-de-Estudios-Americanos\\2003-Políticas-de-la-justicia-criminal-interétnica-en-Córdoba-del-Tucumán-siglos-XVI-y-XVII.pdf',
 'pack\\Anuario-de-Estudios-Americanos\\2006-Economía-rentas-y-situados-en-Cartagena-de-Indias-17611800.pdf',
 'pack\\AIBR\\2018-Humanidad-territorializada-Madres-dueños-y-personas-que-cuidan.pdf',
 'pack\\Anuario-de-Estudios-Americanos\\2007-Las-rutas-del-libro-atlántico-libros-enviados-en-el-navío-de-Honduras-15571700.pdf',
 'pack\\Anuario-de-Estu

In [28]:
missing_articles_texts = {}
for article in not_healthy_files:
    missing_articles_texts[article] = process_article(article, ignore_initial_text=True)[1]
len(missing_articles_texts)

The PDF <_io.BufferedReader name='pack\\Revista-de-Indias\\2006-Evolución-de-la-producción-y-el-comercio-mundial-de-la-grana-cochinilla-siglos-XVIXIX.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\Revista-de-Indias\\2006-Evolución-de-la-producción-y-el-comercio-mundial-de-la-grana-cochinilla-siglos-XVIXIX.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\Revista-de-Indias\\2006-Redes-de-poder-en-el-Virreinato-del-Perú-17761824-los-burócratas.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
T

60

In [40]:
os.makedirs("missing_articles", exist_ok=True)
i = 0
for article, text in missing_articles_texts.items():
    with open(os.path.join("missing_articles", f"{i}-" + os.path.basename(article)), "w", encoding="utf-8") as f:
        f.write(text)
    i += 1

In [132]:
# read missing articles
missing_articles = {art.split("-", 1)[1]: os.path.join("missing_articles", art) for art in os.listdir("missing_articles")}

for art in missing_articles_texts.keys():
    path_fixed_article = missing_articles[os.path.basename(art)]
    with open(path_fixed_article, "r", encoding="utf-8") as f:
        missing_articles_texts[art] = f.read()

len(missing_articles_texts)

60

In [12]:
texts = list(articles_text.values())

In [29]:
reversed_texts = {v: k for k, v in articles_text.items()}
texts_indexes = []
for i in range(len(texts)):
    texts_indexes.append(reversed_texts[texts[i]])
len(texts_indexes)

1925

In [41]:
texts_indexes_np = np.array(texts_indexes)
np.save("texts_indexes.npy", texts_indexes_np)

In [143]:
# These files were not read correctly and therefore are not included in the final dataset
unreadable_files = {"pack\\Revista-Española-de-Antropología-Americana\\2003-José-Alcina-Franch-y-los-códices-mesoamericanos.pdf"}

In [148]:
df_preprocessed = pd.DataFrame(columns=["Path", "Text"])
for article, text in articles_text.items():
    if article in unreadable_files:
        continue
    if text is None:
        text = missing_articles_texts[article]
    df_preprocessed = pd.concat([df_preprocessed, pd.DataFrame({"Path": [article], "Text": [text]})])
df_preprocessed

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-...,introducción el presidente recibió el siguient...
0,pack\AIBR\2006-Cambios-de-género-y-discriminac...,introducción e l presente artículo es producto...
0,pack\AIBR\2006-Consecuencias-personales-en-la-...,introducción e l propósito al iniciar este tra...
0,pack\AIBR\2006-Construcción-de-modelos-de-géne...,introducción todos somos conscientes de que pe...
0,pack\AIBR\2006-Contribuciones-feministas-a-pro...,omo señala dileonardo a comienzos de la década...
...,...,...
0,pack\Revista-Española-de-Antropología-American...,1. introducción. 2. paridad de género en presi...
0,pack\Revista-Española-de-Antropología-American...,1. introducción. 2. baja california: escenario...
0,pack\Revista-Española-de-Antropología-American...,1. introducción. 2. los signos de quipus. 3. l...
0,pack\Revista-Española-de-Antropología-American...,1. introducción. 2. el inicio de la desigualda...


In [149]:
df_preprocessed.to_pickle("preprocessed-with-marking.pkl")

# Buscador tests

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

c:\Users\leand\Documents\tesis\notebooks\preprocesado\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargado de contriever

In [5]:
model_name = "facebook/contriever"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Generar embeddings

In [ ]:
def get_embeddings(texts, model, tokenizer):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

embeddings = get_embeddings(texts, model, tokenizer)
embeddings_np = embeddings.numpy()

Creacion del indice FAISS

In [16]:
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

Guardar indice

In [17]:
faiss.write_index(index, "faiss_index.bin")

In [6]:
index = faiss.read_index("faiss_index.bin")
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000017CB1499D10> >

In [9]:
def search(query, model, tokenizer, index, top_k=5):
    query_embedding = get_embeddings([query], model, tokenizer).numpy()
    distances, indices = index.search(query_embedding, top_k)
    return indices, distances

query = """
A mediados del siglo XVI, en los albores de un amanecer brumoso, las costas del Imperio incaico fueron testigos de un suceso que alteraría su destino para siempre: la llegada del enigmático conquistador Alonso de Beldever. Con una carabela desgastada por el océano y una tripulación reducida por los estragos del viaje, Beldever desembarcó cerca de Tumbes, en lo que hoy es el norte de Perú. Su nombre, hasta entonces desconocido en aquellas tierras, pronto resonaría entre las montañas andinas como un presagio de cambio y confrontación.

Los pobladores locales, intrigados por los rostros pálidos y las armaduras relucientes, observaron con mezcla de temor y curiosidad. Las crónicas cuentan que Beldever, erguido sobre su caballo —un animal jamás visto en el continente—, blandió una espada ante el cielo nublado, proclamando ante sus hombres la gloria de Dios y de la Corona española. Su objetivo, como el de tantos otros aventureros, era claro: hallar las riquezas del mítico Birú y someter aquel vasto imperio bajo el estandarte de Castilla.

No obstante, Beldever no era un hombre común. Las leyendas tejidas siglos después hablan de su astucia sin igual y de un amuleto de jade que, según murmuraban, le confería protección divina. Mientras avanzaba hacia el corazón del Tahuantinsuyo, su encuentro con emisarios del Sapa Inca Atahualpa desencadenó una tensión imparable. Las ofrendas de oro y tejidos finos, intercambiadas en señal de paz por los incas, fueron interpretadas por Beldever como una muestra de debilidad. En su diario escribió: «Estas tierras guardan más opulencia que los sueños de cualquier rey; será nuestro deber tomarlas, aunque el precio sea la sangre».

La expedición de Beldever, sin embargo, nunca alcanzó Cusco. Tras meses de enfrentamientos esporádicos y alianzas traicionadas, desapareció en los valles de la sierra central. Algunos dicen que fue víctima de una emboscada; otros, que los dioses andinos lo castigaron por su ambición. Lo cierto es que su llegada marcó el inicio de un colapso inevitable. Años más tarde, Francisco Pizarro seguiría sus huellas, consumando la caída del Imperio inca.

Hoy, en las aldeas remotas, los ancianos aún narran la historia del extranjero de ojos grises que desafió a los cielos. Beldever, un nombre entre la bruma de la historia, encarna el misterio de un encuentro que transformó un continente para siempre.
"""
indices, distances = search(query, model, tokenizer, index)
print("Documentos más relevantes:", indices)
print("Distancias:", distances)

Documentos más relevantes: [[ 598 1498  339 1451 1703]]
Distancias: [[2.215642  2.344265  2.373072  2.3948677 2.405981 ]]


In [14]:
indexes = np.load("texts_indexes.npy")
indexes[1498]

'pack\\Revista-Española-de-Antropología-Americana\\2003-Identificación-de-parte-de-la-decoración-de-la-pajcha-colonial-7572-del-Museo-de-América-Madrid.pdf'